In [ ]:
#first 
import pandas as pd
from pulp import LpProblem, LpMinimize, LpMaximize, LpVariable, lpSum, value
import json
from io import StringIO

# Load the data from JSON
print("Loading data from 'employes.json'...")
try:
    # Assuming the JSON data is provided as a file
    with open('employees.json', 'r') as file:
        json_data = json.load(file)
    data = pd.DataFrame(json_data)
except FileNotFoundError:
    raise FileNotFoundError("The file 'employees.json' was not found.")
except json.JSONDecodeError:
    raise ValueError("Invalid JSON format in 'employees.json'.")
except Exception as e:
    raise ValueError(f"Error loading JSON data: {str(e)}")
print("Data loaded successfully.")

# Validate required columns
required_columns = [
    'employeeId', 'memberStatus', 'employerContribution', 'premium', 'composition',
    'plan_level_id3', 'plan_level_id3_premium', 'hsa_eligible_plan',
    'hsa_eligible_plan_premium', 'reco_plan1', 'reco_plan1_premium',
    'reco_plan3', 'reco_plan3_premium', 'reco_plan4', 'reco_plan4_premium'
]
missing_columns = [col for col in required_columns if col not in data.columns]
if missing_columns:
    raise ValueError(f"Missing required columns: {missing_columns}")
print("All required columns validated.")

# Initialize variables
employees = data['employeeId'].astype(str).tolist()
num_employees = len(employees)
plans = set()
print(f"Number of employees: {num_employees}")

# Map plan types to their premium columns
plan_types = {
    'plan_level_id3': 'plan_level_id3_premium',
    'hsa_eligible_plan': 'hsa_eligible_plan_premium',
    'reco_plan1': 'reco_plan1_premium',
    'reco_plan3': 'reco_plan3_premium',
    'reco_plan4': 'reco_plan4_premium'
}
print("Plan types mapped to premium columns.")

# Get unique member status and composition combinations and current contributions
status_composition_pairs = data[['memberStatus', 'composition']].drop_duplicates().values
current_contributions = {}
for status, comp in status_composition_pairs:
    contrib = data[(data['memberStatus'] == status) & (data['composition'] == comp)]['employerContribution'].iloc[0] / 100.0
    current_contributions[(status, comp)] = contrib
print("Member Status and Composition combinations with contributions calculated:")
for (status, comp), contrib in current_contributions.items():
    print(f"  {status}, {comp}: {contrib*100:.0f}%")

# Function to get user input
def get_user_input():
    print("\nEnter budget (or 'no budget' for default calculation):")
    budget_input = input().strip().lower()
    if budget_input == 'no budget':
        budget = None
    else:
        try:
            budget = float(budget_input)
            if budget <= 0:
                raise ValueError("Budget must be positive.")
        except ValueError:
            raise ValueError("Invalid budget input. Please enter a number or 'no budget'.")
    
    print("Provide contribution set? (yes/no):")
    contrib_input = input().strip().lower()
    if contrib_input == 'yes':
        contrib_set = {}
        print("Enter contribution percentages for each member status and composition (as decimals, e.g., 0.5 for 50%):")
        for status, comp in status_composition_pairs:
            print(f"Contribution for {status}, {comp}:")
            try:
                contrib = float(input())
                if contrib < 0 or contrib > 1:
                    raise ValueError(f"Contribution for {status}, {comp} must be between 0 and 1.")
                contrib_set[(status, comp)] = contrib
            except ValueError:
                raise ValueError(f"Invalid contribution for {status}, {comp}. Please enter a number between 0 and 1.")
    else:
        contrib_set = current_contributions.copy()
    
    return budget, contrib_set

# Function to run optimization
def run_optimization(contrib_set, budget, objective_type, label, fp1_assignments=None):
    print(f"\n=== Running Optimization: {label} ===")
    print("Contribution Set:")
    for (status, comp), contrib in contrib_set.items():
        print(f"  {status}, {comp}: {contrib*100:.0f}%")
    if budget is not None:
        print(f"Budget: ${budget:.2f}")
    else:
        print("Budget: None (No constraint)")
    
    costs = []
    preferences = []
    preference_scores = []
    for idx, row in data.iterrows():
        emp_costs = {}
        emp_scores = {}
        status = row['memberStatus']
        comp = row['composition']
        contrib = contrib_set.get((status, comp), 0.0)
        
        for plan_type, premium_col in plan_types.items():
            plan_name = row[plan_type]
            premium = row[premium_col]
            if pd.notna(plan_name) and pd.notna(premium):
                plans.add(str(plan_name))  # Ensure plan_name is string
                emp_costs[str(plan_name)] = float(premium) * contrib
        
        for plan_type in plan_types:
            plan_name = row[plan_type]
            if pd.notna(plan_name):
                plan_name = str(plan_name)
                if plan_name not in emp_scores:
                    emp_scores[plan_name] = 0
                if plan_type == 'hsa_eligible_plan':
                    emp_scores[plan_name] += 10
                elif plan_type == 'reco_plan1':
                    emp_scores[plan_name] += 8
                elif plan_type == 'reco_plan4':
                    emp_scores[plan_name] += 7
                elif plan_type == 'reco_plan3':
                    emp_scores[plan_name] += 6
                else:
                    emp_scores[plan_name] += 5
        
        sorted_plans = sorted(emp_scores.items(), key=lambda x: x[1], reverse=True)
        preferences.append([plan[0] for plan in sorted_plans])
        preference_scores.append({plan: score for plan, score in emp_scores.items()})
        costs.append(emp_costs)
    
    plans_list = list(plans)
    num_plans = len(plans_list)
    if num_plans == 0:
        raise ValueError("No valid plans found in the data.")
    plan_indices = {plan: idx for idx, plan in enumerate(plans_list)}
    
    print("Available Plans:", plans_list)
    print("\nCosts per Employee:")
    for i, emp_costs in enumerate(costs):
        print(f"  Employee {employees[i]}: {emp_costs}")
    print("\nPreferences per Employee:")
    for i, pref in enumerate(preferences):
        print(f"  Employee {employees[i]}: {pref}")
    
    problem = LpProblem(f"Employee_Plan_Assignment_{label}", LpMinimize if objective_type == 'cost' else LpMaximize)
    
    x = [[LpVariable(f"x_{i}_{j}", cat="Binary") for j in range(num_plans)] for i in range(num_employees)]
    y = [LpVariable(f"y_{j}", cat="Binary") for j in range(num_plans)]
    
    # Objective
    if objective_type == 'cost':
        problem += lpSum(costs[i][plan] * x[i][plan_indices[plan]]
                         for i in range(num_employees)
                         for plan in costs[i]), "Minimize_Cost"
    else:
        problem += lpSum(preference_scores[i][plan] * x[i][plan_indices[plan]]
                         for i in range(num_employees)
                         for plan in preference_scores[i]), "Maximize_Preference"
    
    # Constraint: Each employee gets exactly one plan
    for i in range(num_employees):
        problem += lpSum(x[i][j] for j in range(num_plans)) == 1, f"One_Plan_Employee_{i}"
    
    # Constraint: Employees can only be assigned available plans
    for i in range(num_employees):
        for j in range(num_plans):
            plan = plans_list[j]
            if plan not in costs[i]:
                problem += x[i][j] == 0, f"No_Plan_{i}_{plan}"
    
    # Constraint: At least one HSA-eligible plan assigned (if available)
    hsa_plans = [plan for plan in plans_list if plan in data['hsa_eligible_plan'].dropna().astype(str).unique()]
    if hsa_plans:
        problem += lpSum(x[i][plan_indices[plan]] for i in range(num_employees) for plan in hsa_plans) >= 1, "HSA_Requirement"
    
    # Constraint: At least one Level3 plan assigned (if available)
    level3_plans = [plan for plan in plans_list if plan in data['plan_level_id3'].dropna().astype(str).unique()]
    if level3_plans:
        problem += lpSum(x[i][plan_indices[plan]] for i in range(num_employees) for plan in level3_plans) >= 1, "Level3_Requirement"
    
    # Budget constraint (if applicable)
    if budget is not None:
        problem += lpSum(costs[i][plan] * x[i][plan_indices[plan]]
                         for i in range(num_employees)
                         for plan in costs[i]) <= budget, "Budget_Constraint"
    
    # Constraint: Maximum 5 unique plans assigned
    for j in range(num_plans):
        problem += lpSum(x[i][j] for i in range(num_employees)) <= num_employees * y[j], f"Plan_Used_{j}"
    problem += lpSum(y[j] for j in range(num_plans)) <= 5, "Max_Unique_Plans"
    
    # Constraint: fp2 must differ from fp1 (if fp1_assignments provided)
    if fp1_assignments and objective_type == 'preference':
        print("Adding constraint to ensure fp2 differs from fp1...")
        diff_vars = [LpVariable(f"diff_{i}", cat="Binary") for i in range(num_employees)]
        for i in range(num_employees):
            fp1_plan = next(plan for emp, plan in fp1_assignments if emp == employees[i])
            fp1_plan_idx = plan_indices[fp1_plan]
            problem += diff_vars[i] >= 1 - x[i][fp1_plan_idx], f"Diff_Constraint_{i}"
        problem += lpSum(diff_vars[i] for i in range(num_employees)) >= 1, "Different_From_fp1"
    
    print(f"Solving optimization problem for {label}...")
    status = problem.solve()
    
    if status != 1:
        print(f"Solver failed with status: {status}. Possible infeasible problem.")
        return None
    
    assignments = []
    total_cost = 0
    total_preference = 0
    for i in range(num_employees):
        for j in range(num_plans):
            if value(x[i][j]) > 0.5:
                plan = plans_list[j]
                if plan not in costs[i]:
                    print(f"Error: Employee {employees[i]} assigned unavailable plan {plan}")
                    return None
                assignments.append((employees[i], plan))
                total_cost += costs[i][plan]
                total_preference += preference_scores[i].get(plan, 0)
                break
        else:
            print(f"Employee {employees[i]} was not assigned a plan.")
            return None
    
    hsa_assigned = any(plan in hsa_plans for _, plan in assignments)
    
    print("\nEmployee Plan Assignments:")
    for emp_id, plan in assignments:
        print(f"  Employee {emp_id}: {plan}")
    print(f"\nTotal Employer Cost: ${total_cost:.2f}")
    print(f"Total Preference Score: {total_preference}")
    print(f"At least one HSA-eligible plan assigned: {hsa_assigned}")
    
    unique_plans = len(set(plan for _, plan in assignments))
    print(f"Number of unique plans assigned: {unique_plans}")
    
    return {
        'assignments': assignments,
        'total_cost': total_cost,
        'total_preference': total_preference,
        'hsa_assigned': hsa_assigned
    }

# Main execution
print("\n=== Starting Employee Plan Optimization ===")
try:
    budget, contrib_set = get_user_input()
except ValueError as e:
    print(f"Error: {e}")
    exit(1)

# Calculate default budget if none provided
if budget is None:
    budget = sum(row['premium'] * contrib_set.get((row['memberStatus'], row['composition']), 0.0)
                 for _, row in data.iterrows())
    print(f"No budget specified. Calculated default budget: ${budget:.2f}")
else:
    print(f"User-specified budget: ${budget:.2f}")

# Run optimizations
results = []
# Run for lowest cost (fp1 plans)
print("\n=== Optimizing for Lowest Total Cost (fp1 plans) ===")
fp1_result = run_optimization(contrib_set, budget, 'cost', 'fp1_Lowest_Cost')
if fp1_result:
    results.append(('fp1_Lowest_Cost', budget, fp1_result))
    fp1_assignments = fp1_result['assignments']
else:
    fp1_assignments = None

# Run for highest preference (fp2 plans), excluding fp1 solution
print("\n=== Optimizing for Highest Preference (fp2 plans) ===")
fp2_result = run_optimization(contrib_set, budget, 'preference', 'fp2_Highest_Preference', fp1_assignments)
if fp2_result:
    results.append(('fp2_Highest_Preference', budget, fp2_result))

# Summary
print("\n=== Summary of Results ===")
for label, budget_val, result in results:
    print(f"\nOptimization: {label}")
    print("Contribution Set:")
    for (status, comp), contrib in contrib_set.items():
        print(f"  {status}, {comp}: {contrib*100:.0f}%")
    if budget_val is not None:
        print(f"Budget: ${budget_val:.2f}")
    else:
        print("Budget: None")
    print("Assignments:")
    for emp_id, plan in result['assignments']:
        print(f"  Employee {emp_id}: {plan}")
    print(f"Total Cost: ${result['total_cost']:.2f}")
    print(f"Total Preference Score: {result['total_preference']}")
    print(f"HSA Assigned: {result['hsa_assigned']}")
    unique_plans = len(set(plan for _, plan in result['assignments']))
    print(f"Number of Unique Plans: {unique_plans}")
print("\n=== Optimization Complete ===")

Loading data from 'employes.json'...
Data loaded successfully.
All required columns validated.
Number of employees: 7
Plan types mapped to premium columns.
Member Status and Composition combinations with contributions calculated:
  FT, Employee Only: 85%
  FT, Employee + Spouse: 75%
  FT, Employee + Family: 65%

=== Starting Employee Plan Optimization ===

Enter budget (or 'no budget' for default calculation):
Provide contribution set? (yes/no):
Enter contribution percentages for each member status and composition (as decimals, e.g., 0.5 for 50%):
Contribution for FT, Employee Only:
Contribution for FT, Employee + Spouse:
Contribution for FT, Employee + Family:
No budget specified. Calculated default budget: $6748.09

=== Optimizing for Lowest Total Cost (fp1 plans) ===

=== Running Optimization: fp1_Lowest_Cost ===
Contribution Set:
  FT, Employee Only: 55%
  FT, Employee + Spouse: 60%
  FT, Employee + Family: 70%
Budget: $6748.09
Available Plans: ['58081GA0010075', '58081GA0010030', 

In [ ]:
#first with excel output
import pandas as pd
from pulp import LpProblem, LpMinimize, LpMaximize, LpVariable, lpSum, value
import json
import logging
import time

# Configure logging
logging.basicConfig(
    format='%(asctime)s [%(levelname)s] %(message)s',
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger(__name__)

# Global debug flag
debug = 1
logger.setLevel(logging.DEBUG)
for handler in logger.handlers:
    handler.setLevel(logging.DEBUG)

# Initialize Excel writer
excel_writer = pd.ExcelWriter('output.xlsx', engine='openpyxl')

# Load and validate data
def load_and_validate_data(file_path):
    if debug:
        print("\nHey! We're loading a list of employees and their health plan options from a file called 'employees.json'. "
              "This is like getting a class roster with everyone's favorite activities!")
    logger.info(f"Loading data from '{file_path}'...")
    try:
        with open(file_path, 'r') as file:
            json_data = json.load(file)
        data = pd.DataFrame(json_data)
    except FileNotFoundError:
        logger.error(f"The file '{file_path}' was not found.")
        raise
    except json.JSONDecodeError:
        logger.error(f"Invalid JSON format in '{file_path}'.")
        raise
    except Exception as e:
        logger.error(f"Error loading JSON data: {str(e)}")
        raise
    logger.info("Data loaded successfully.")

    if debug:
        print("Checking if the file has all the info we need, like employee IDs and plan costs!")
    required_columns = [
        'employeeId', 'memberStatus', 'composition', 'premium', 'plan_level_id3', 'plan_level_id3_premium',
        'hsa_eligible_plan', 'hsa_eligible_plan_premium', 'reco_plan1', 'reco_plan1_premium',
        'reco_plan3', 'reco_plan3_premium', 'reco_plan4', 'reco_plan4_premium'
    ]
    missing_columns = [col for col in required_columns if col not in data.columns]
    if missing_columns:
        logger.error(f"Missing required columns: {missing_columns}")
        raise ValueError(f"Missing required columns: {missing_columns}")
    logger.info("All required columns validated.")

    if debug:
        print("Ensuring at least one HSA plan exists, like checking for a goalie!")
    if data['hsa_eligible_plan'].isna().all():
        logger.error("No HSA-eligible plans found.")
        raise ValueError("No HSA-eligible plans found.")
    logger.info("HSA-eligible plan validation passed.")

    data.to_excel(excel_writer, sheet_name='Load_Validate_Data', index=False)
    sheet = excel_writer.sheets['Load_Validate_Data']
    sheet['A' + str(len(data) + 3)] = (
        "Explanation: This table shows all employee info, like IDs and plan options. "
        "It’s like a roster telling us who’s playing and what plans they can pick!"
    )
    logger.debug("Saved load_and_validate_data output to Excel")

    return data

# Initialize variables
def initialize_variables(data):
    if debug:
        print("\nSetting up the basics: employees, plans, and contribution groups!")
    employees = data['employeeId'].astype(str).tolist()
    num_employees = len(employees)
    plans = set()
    plan_types = {
        'plan_level_id3': 'plan_level_id3_premium',
        'hsa_eligible_plan': 'hsa_eligible_plan_premium',
        'reco_plan1': 'reco_plan1_premium',
        'reco_plan3': 'reco_plan3_premium',
        'reco_plan4': 'reco_plan4_premium'
    }
    status_composition_pairs = data[['memberStatus', 'composition']].drop_duplicates().values.tolist()
    logger.info(f"Initialized: {num_employees} employees, {len(status_composition_pairs)} status-composition pairs")
    if debug:
        print(f"Found {num_employees} employees and {len(status_composition_pairs)} groups for contributions!")

    init_data = {
        'Number of Employees': [num_employees],
        'Plan Types': [str(plan_types)],
        'Status-Composition Pairs': [str(status_composition_pairs)]
    }
    init_df = pd.DataFrame(init_data)
    init_df.to_excel(excel_writer, sheet_name='Initialize_Variables', index=False)
    sheet = excel_writer.sheets['Initialize_Variables']
    sheet['A4'] = (
        "Explanation: This table sets up our game board: how many employees, plan types, and groups for contributions. "
        "It’s like listing players, games, and teams!"
    )
    logger.debug("Saved initialize_variables output to Excel")

    return employees, num_employees, plans, plan_types, status_composition_pairs

# Compute contributions (for default use)
def compute_contributions(data, status_composition_pairs):
    if debug:
        print("\nFiguring out default company payments for each employee group!")
    contributions = {}
    for status, comp in status_composition_pairs:
        contrib = data[(data['memberStatus'] == status) & (data['composition'] == comp)]['employerContribution'].iloc[0] / 100.0
        contributions[(status, comp)] = contrib
        logger.debug(f"Set default contribution for {status}, {comp}: {contrib*100:.0f}%")
    logger.info("Default contributions calculated")
    if debug:
        print("Made a default list of how much the company pays for each group!")

    contrib_data = [{'Status': status, 'Composition': comp, 'Contribution': contrib*100} for (status, comp), contrib in contributions.items()]
    contrib_df = pd.DataFrame(contrib_data)
    contrib_df.to_excel(excel_writer, sheet_name='Compute_Contributions', index=False)
    sheet = excel_writer.sheets['Compute_Contributions']
    sheet['A' + str(len(contrib_data) + 3)] = (
        "Explanation: This table shows default company payments for each employee group. "
        "It’s like deciding how many snacks each team gets!"
    )
    logger.debug("Saved compute_contributions output to Excel")

    return contributions

# Process employee data
def process_employee_data(data, plan_types, plans, status_composition_pairs):
    if debug:
        print("\nChecking each employee’s plan options, costs, and preferences!")
    premiums = []
    preferences = []
    preference_scores = []
    employee_groups = []
    logger.info("Processing employee data")
    for idx, row in data.iterrows():
        emp_premiums = {}
        emp_scores = {}
        status = row['memberStatus']
        comp = row['composition']
        employee_groups.append((status, comp))
        logger.debug(f"Employee {row['employeeId']}: status={status}, composition={comp}")

        for plan_type, premium_col in plan_types.items():
            plan_name = row[plan_type]
            premium = row[premium_col]
            if pd.notna(plan_name) and pd.notna(premium):
                plans.add(str(plan_name))
                emp_premiums[str(plan_name)] = float(premium)
                logger.debug(f"  Added plan {plan_name} with premium {premium:.2f}")
            else:
                logger.debug(f"  Skipping plan {plan_type}")

        for plan_type in plan_types:
            plan_name = row[plan_type]
            if pd.notna(plan_name):
                plan_name = str(plan_name)
                if plan_name not in emp_scores:
                    emp_scores[plan_name] = 0
                if plan_type == 'hsa_eligible_plan':
                    emp_scores[plan_name] += 10
                elif plan_type == 'reco_plan1':
                    emp_scores[plan_name] += 8
                elif plan_type == 'reco_plan4':
                    emp_scores[plan_name] += 7
                elif plan_type == 'reco_plan3':
                    emp_scores[plan_name] += 6
                else:
                    emp_scores[plan_name] += 5
                logger.debug(f"  Plan {plan_name} score: {emp_scores[plan_name]}")

        sorted_plans = sorted(emp_scores.items(), key=lambda x: x[1], reverse=True)
        preferences.append([plan[0] for plan in sorted_plans])
        preference_scores.append({plan: score for plan, score in emp_scores.items()})
        premiums.append(emp_premiums)
        logger.debug(f"Employee {row['employeeId']}: Premiums={emp_premiums}, Preferences={preferences[-1]}")
    if debug:
        print("Done! We know each employee’s plan costs and preferences!")

    premiums_data = [{'Employee': f'Emp_{i}', 'Plan': plan, 'Premium': premium} for i, emp_premiums in enumerate(premiums) for plan, premium in emp_premiums.items()]
    prefs_data = [{'Employee': f'Emp_{i}', 'Preferences': ', '.join(prefs)} for i, prefs in enumerate(preferences)]
    prefs_df = pd.DataFrame(prefs_data)
    scores_data = [{'Employee': f'Emp_{i}', 'Plan': plan, 'Score': score} for i, emp_scores in enumerate(preference_scores) for plan, score in emp_scores.items()]
    scores_df = pd.DataFrame(scores_data)
    groups_data = [{'Employee': f'Emp_{i}', 'Status': status, 'Composition': comp} for i, (status, comp) in enumerate(employee_groups)]
    groups_df = pd.DataFrame(groups_data)
    premiums_df = pd.DataFrame(premiums_data)
    premiums_df.to_excel(excel_writer, sheet_name='Process_Employee_Data_Premiums', index=False)
    prefs_df.to_excel(excel_writer, sheet_name='Process_Employee_Data_Prefs', index=False)
    scores_df.to_excel(excel_writer, sheet_name='Process_Employee_Data_Scores', index=False)
    groups_df.to_excel(excel_writer, sheet_name='Process_Employee_Data_Groups', index=False)
    sheet = excel_writer.sheets['Process_Employee_Data_Premiums']
    sheet['A' + str(len(premiums_data) + 3)] = (
        "Explanation: This table shows the base cost (premium) of each plan for each employee. "
        "It’s like a price list for games!"
    )
    sheet = excel_writer.sheets['Process_Employee_Data_Prefs']
    sheet['A' + str(len(prefs_data) + 3)] = (
        "Explanation: This table lists each employee’s preferred plans, in order. "
        "It’s like ranking their favorite games!"
    )
    sheet = excel_writer.sheets['Process_Employee_Data_Scores']
    sheet['A' + str(len(scores_data) + 3)] = (
        "Explanation: This table shows how much each employee likes each plan. "
        "Higher scores mean they love it, like a 10/10 game!"
    )
    sheet = excel_writer.sheets['Process_Employee_Data_Groups']
    sheet['A' + str(len(groups_data) + 3)] = (
        "Explanation: This table shows which group each employee belongs to for contributions. "
        "It’s like sorting players into teams!"
    )
    logger.debug("Saved process_employee_data output to Excel")

    return premiums, preferences, preference_scores, employee_groups

# Setup LP model
def setup_lp_model(plans, num_employees, status_composition_pairs, objective_type, label):
    if debug:
        print("\nSetting up the puzzle board with employees, plans, and contribution rates!")
    plans_list = list(plans)
    num_plans = len(plans_list)
    if num_plans == 0:
        logger.error("No valid plans found.")
        raise ValueError("No valid plans found.")
    plan_indices = {plan: idx for idx, plan in enumerate(plans_list)}

    problem = LpProblem(f"Employee_Plan_Assignment_{label}", LpMinimize if objective_type == 'cost' else LpMaximize)
    logger.info(f"Created LP problem: {problem.name}")

    x = [[LpVariable(f"x_{i}_{j}", cat="Binary") for j in range(num_plans)] for i in range(num_employees)]
    y = [LpVariable(f"y_{j}", cat="Binary") for j in range(num_plans)]
    contrib_vars = {
        (status, comp): LpVariable(f"contrib_{status}_{comp}", lowBound=0.1, upBound=1.0, cat="Continuous")
        for status, comp in status_composition_pairs
    }
    # Auxiliary variables for cost: z_{i,j,status,comp} = premium_j * contrib_{status,comp} * x_{i,j}
    z_vars = {
        (i, j, status, comp): LpVariable(f"z_{i}_{j}_{status}_{comp}", lowBound=0, cat="Continuous")
        for i in range(num_employees)
        for j in range(num_plans)
        for status, comp in status_composition_pairs
    }
    logger.debug(f"Defined {num_employees * num_plans} assignment vars, {num_plans} plan usage vars, "
                 f"{len(contrib_vars)} contribution vars, {len(z_vars)} auxiliary vars")
    if debug:
        print(f"Made a grid for {num_employees} employees, {num_plans} plans, {len(contrib_vars)} contribution rates, "
              f"and {len(z_vars)} cost helpers!")

    lp_data = {
        'Objective': [objective_type],
        'Number of Employees': [num_employees],
        'Number of Plans': [num_plans],
        'Assignment Variables': [num_employees * num_plans],
        'Plan Usage Variables': [num_plans],
        'Contribution Variables': [len(contrib_vars)],
        'Auxiliary Variables': [len(z_vars)]
    }
    lp_df = pd.DataFrame(lp_data)
    lp_df.to_excel(excel_writer, sheet_name=f'Setup_LP_Model_{label}', index=False)
    sheet = excel_writer.sheets[f'Setup_LP_Model_{label}']
    sheet['A8'] = (
        "Explanation: This table sets up our puzzle with employees, plans, contribution rates, and cost helpers. "
        "It’s like counting players, games, team budgets, and score trackers!"
    )
    logger.debug(f"Saved setup_lp_model output for {label}")

    return problem, x, y, contrib_vars, z_vars, plans_list, plan_indices

# Set objective function
def set_objective(problem, objective_type, premiums, preference_scores, x, contrib_vars, z_vars, plan_indices, employees, employee_groups):
    if debug:
        print("\nDeciding our goal: save money or make employees happy, while setting contribution rates!")
    logger.info(f"Constructing objective ({objective_type})...")
    if objective_type == 'cost':
        objective_terms = []
        for i in range(len(employees)):
            status, comp = employee_groups[i]
            for plan in premiums[i]:
                j = plan_indices[plan]
                # Use z_vars for cost: premium_j * contrib_{status,comp} * x_{i,j}
                objective_terms.append(premiums[i][plan] * z_vars[(i, j, status, comp)])
                logger.debug(f"  Employee {employees[i]}, Plan {plan}, Premium {premiums[i][plan]:.2f}")
        problem += lpSum(objective_terms), "Minimize_Cost"
        logger.info(f"Objective: Minimize {len(objective_terms)} cost terms")
        if debug:
            print("We’re trying to spend as little as possible while picking plans and contribution rates!")
    else:
        objective_terms = []
        for i in range(len(employees)):
            for plan in preference_scores[i]:
                term = preference_scores[i][plan] * x[i][plan_indices[plan]]
                objective_terms.append(term)
                logger.debug(f"  Employee {employees[i]}, Plan {plan}, Score {preference_scores[i][plan]}")
        problem += lpSum(objective_terms), "Maximize_Preference"
        logger.info(f"Objective: Maximize {len(objective_terms)} preference terms")
        if debug:
            print("We’re making employees as happy as possible with their favorite plans and fair contributions!")

    obj_data = {
        'Objective Type': [objective_type],
        'Number of Terms': [len(objective_terms)]
    }
    obj_df = pd.DataFrame(obj_data)
    obj_df.to_excel(excel_writer, sheet_name=f'Set_Objective_{objective_type}', index=False)
    sheet = excel_writer.sheets[f'Set_Objective_{objective_type}']
    sheet['A4'] = (
        f"Explanation: We’re {'saving money' if objective_type == 'cost' else 'making employees happy'}, "
        "while setting contribution rates. It’s like choosing cheap or fun games!"
    )
    logger.debug(f"Saved set_objective output for {objective_type}")

# Add constraints
def add_constraints(problem, x, y, contrib_vars, z_vars, employees, premiums, plan_indices, plans_list, hsa_plans, data, budget, num_employees, num_plans, employee_groups, fp1_assignments, objective_type):
    if debug:
        print("\nAdding rules to keep our assignments and contributions fair!")
    constraints_added = []

    # One plan per employee
    if debug:
        print("Rule 1: Every employee gets one plan!")
    for i in range(num_employees):
        problem += lpSum([x[i][j] for j in range(num_plans)]) == 1, f"One_Plan_Employee_{i}"
    constraints_added.append({
        'Constraint': 'One plan per employee',
        'Formula': r'\sum_{j} x_{i,j} = 1 \quad \forall i',
        'Example': 'For emp123, x_{emp123,HSA_Plan_1} + x_{emp123,Plan_Level3_1} = 1, like picking one game!'
    })

    # Available plans only
    if debug:
        print("Rule 2: Only assign available plans!")
    for i in range(num_employees):
        for j in range(num_plans):
            plan = plans_list[j]
            if plan not in premiums[i]:
                problem += x[i][j] == 0, f"No_Plan_{i}_{plan}"
    constraints_added.append({
        'Constraint': 'Available plans only',
        'Formula': r'x_{i,j} = 0 \quad \forall i, j \text{ where plan } j \notin \text{premiums}_i',
        'Example': 'If emp123 can’t pick HSA_Plan_1, x_{emp123,HSA_Plan_1} = 0, like skipping a game they don’t have!'
    })

    # Mandatory HSA plan
    if debug:
        print("Rule 3: At least one HSA plan!")
    hsa_terms = []
    for i in range(num_employees):
        for plan in hsa_plans:
            hsa_terms.append(x[i][plan_indices[plan]])
    problem += lpSum(hsa_terms) >= 1, "HSA_Requirement"
    constraints_added.append({
        'Constraint': 'Mandatory HSA plan',
        'Formula': r'\sum_{i} \sum_{j \in \text{HSA_plans}} x_{i,j} \geq 1',
        'Example': 'x_{emp123,HSA_Plan_1} + x_{emp456,HSA_Plan_1} \geq 1, like needing one goalie!'
    })

    # Level3 plan (if available)
    if debug:
        print("Rule 4: At least one Level3 plan if available!")
    level3_plans = [plan for plan in plans_list if plan in data['plan_level_id3'].dropna().astype(str).unique()]
    if level3_plans:
        level3_terms = []
        for i in range(num_employees):
            for plan in level3_plans:
                level3_terms.append(x[i][plan_indices[plan]])
        problem += lpSum(level3_terms) >= 1, "Level3_Requirement"
        constraints_added.append({
            'Constraint': 'Level3 plan',
            'Formula': r'\sum_{i} \sum_{j \in \text{Level3_plans}} x_{i,j} \geq 1 \text{ if Level3_plans exist}',
            'Example': 'x_{emp123,Plan_Level3_1} + x_{emp456,Plan_Level3_1} \geq 1, like trying a deluxe game!'
        })
    else:
        constraints_added.append({
            'Constraint': 'Level3 plan (skipped)',
            'Formula': 'Not applicable',
            'Example': 'No Level3 plans, so we skip this, like no deluxe games available!'
        })

    # Budget constraint
    if debug:
        print("Rule 5: Stay within budget using contribution rates!")
    if budget is not None:
        budget_terms = []
        for i in range(num_employees):
            status, comp = employee_groups[i]
            for plan in premiums[i]:
                j = plan_indices[plan]
                budget_terms.append(premiums[i][plan] * z_vars[(i, j, status, comp)])
        problem += lpSum(budget_terms) <= budget, "Budget_Constraint"
        constraints_added.append({
            'Constraint': 'Budget constraint',
            'Formula': r'\sum_{i} \sum_{j} \text{premium}_{j} \cdot z_{i,j,\text{status,comp}} \leq \text{budget}',
            'Example': '5000 \cdot z_{emp123,HSA_Plan_1,Single,Employee} + ... \leq 10000, like staying under arcade money!'
        })
    else:
        constraints_added.append({
            'Constraint': 'Budget constraint (skipped)',
            'Formula': 'Not applicable',
            'Example': 'No budget set, like unlimited arcade tokens!'
        })

    # Maximum 5 unique plans
    if debug:
        print("Rule 6: Use up to 5 different plans!")
    for j in range(num_plans):
        problem += lpSum([x[i][j] for i in range(num_employees)]) <= num_employees * y[j], f"Plan_Used_{j}"
    problem += lpSum([y[j] for j in range(num_plans)]) <= 5, "Max_Unique_Plans"
    constraints_added.append({
        'Constraint': 'Maximum 5 unique plans',
        'Formula': r'\sum_{j} y_j \leq 5, \quad \sum_{i} x_{i,j} \leq M \cdot y_j \quad \forall j',
        'Example': 'y_{HSA_Plan_1} + y_{Plan_Level3_1} \leq 5, like picking 5 games!'
    })

    # fp2 differs from fp1
    if fp1_assignments and objective_type == 'preference':
        if debug:
            print("Rule 7: fp2 differs from fp1!")
        diff_vars = [LpVariable(f"diff_{i}", cat="Binary") for i in range(num_employees)]
        for i in range(num_employees):
            fp1_plan = next(plan for emp, plan in fp1_assignments if emp == employees[i])
            problem += diff_vars[i] >= 1 - x[i][plan_indices[fp1_plan]], f"Diff_Constraint_{i}"
        problem += lpSum(diff_vars) >= 1, "Different_From_fp1"
        constraints_added.append({
            'Constraint': 'fp2 differs from fp1',
            'Formula': r'\sum_{i} \text{diff}_i \geq 1, \quad \text{diff}_i \geq 1 - x_{i,j_{\text{fp1}}} \quad \forall i',
            'Example': 'diff_{emp123} \geq 1 - x_{emp123,HSA_Plan_1}, sum diff_i \geq 1, like switching teams!'
        })

    # Contribution bounds (updated to 10%–100%)
    if debug:
        print("Rule 8: Contributions between 10% and 100%!")
    constraints_added.append({
        'Constraint': 'Contribution bounds',
        'Formula': r'0.1 \leq \text{contrib}_{\text{status,comp}} \leq 1.0 \quad \forall \text{(status,comp)}',
        'Example': 'contrib_{Single,Employee} between 0.1 and 1.0, like setting a fair share of arcade tokens from 10% to 100%!'
    })

    # Auxiliary variable constraints for z_{i,j,status,comp}
    if debug:
        print("Rule 9: Link contributions and assignments for costs!")
    for i in range(num_employees):
        status, comp = employee_groups[i]
        for j in range(num_plans):
            plan = plans_list[j]
            if plan in premiums[i]:
                z = z_vars[(i, j, status, comp)]
                contrib = contrib_vars[(status, comp)]
                x_ij = x[i][j]
                # Linear constraints to enforce z = contrib * x_ij
                problem += z <= contrib, f"Z_Upper_Bound_{i}_{j}_{status}_{comp}"
                problem += z <= x_ij, f"Z_Assignment_Bound_{i}_{j}_{status}_{comp}"
                problem += z >= contrib - (1 - x_ij), f"Z_Lower_Bound_{i}_{j}_{status}_{comp}"
                problem += z >= 0, f"Z_Nonnegative_{i}_{j}_{status}_{comp}"
            else:
                # If plan is not available, z = 0
                problem += z_vars[(i, j, status, comp)] == 0, f"Z_Unavailable_{i}_{j}_{status}_{comp}"
    constraints_added.append({
        'Constraint': 'Auxiliary cost variables',
        'Formula': r'z_{i,j,s,c} \leq \text{contrib}_{s,c}, \quad z_{i,j,s,c} \leq x_{i,j}, \quad z_{i,j,s,c} \geq \text{contrib}_{s,c} - (1 - x_{i,j}), \quad z_{i,j,s,c} \geq 0',
        'Example': 'z_{emp123,HSA_Plan_1,Single,Employee} links contrib_{Single,Employee} and x_{emp123,HSA_Plan_1}, like tracking game costs!'
    })

    constraints_df = pd.DataFrame(constraints_added)
    constraints_df.to_excel(excel_writer, sheet_name=f'Add_Constraints_{objective_type}', index=False)
    sheet = excel_writer.sheets[f'Add_Constraints_{objective_type}']
    sheet['A' + str(len(constraints_added) + 3)] = (
        "Explanation: This table lists rules for fair assignments and contributions. "
        "Each rule has a formula and example, like game rules!"
    )
    logger.debug(f"Saved add_constraints output for {objective_type}")

# Solve LP model
def solve_model(problem, label):
    if debug:
        print("\nSolving the puzzle with a smart computer trick!")
    logger.info(f"Solving for {label}...")
    start_time = time.time()
    status = problem.solve()
    solve_time = time.time() - start_time
    logger.info(f"Solver status: {status}, time: {solve_time:.2f}s")
    if debug:
        print(f"Solved in {solve_time:.2f} seconds, status {status}!")

    solve_data = {
        'Label': [label],
        'Status': [status],
        'Solve Time (seconds)': [solve_time]
    }
    solve_df = pd.DataFrame(solve_data)
    solve_df.to_excel(excel_writer, sheet_name=f'Solve_Model_{label}', index=False)
    sheet = excel_writer.sheets[f'Solve_Model_{label}']
    sheet['A4'] = (
        "Explanation: Shows if we solved the puzzle and how long it took. "
        "Status 1 means we won!"
    )
    logger.debug(f"Saved solve_model output for {label}")

    return status

# Process results
def process_results(status, x, y, contrib_vars, z_vars, employees, premiums, preference_scores, plans_list, plan_indices, num_employees, num_plans, hsa_plans, data, budget, fp1_assignments, objective_type, label, employee_groups):
    if debug:
        print("\nChecking who got which plan and what contribution rates we set!")
    if status != 1:
        logger.warning(f"Solver failed: {status}")
        print(f"Solver failed: {status}")
        return None

    assignments = []
    contributions = {}
    total_cost = 0
    total_preference = 0
    constraint_status = {}
    logger.info("Processing assignments and contributions")

    # Extract assignments and compute costs using z_vars
    for i in range(num_employees):
        assigned = False
        for j in range(num_plans):
            if value(x[i][j]) > 0.5:
                plan = plans_list[j]
                if plan not in premiums[i]:
                    logger.error(f"Employee {employees[i]} assigned unavailable plan {plan}")
                    print(f"Error: Employee {employees[i]} assigned unavailable plan")
                    return None
                assignments.append((employees[i], plan))
                status, comp = employee_groups[i]
                z = z_vars[(i, j, status, comp)]
                cost = premiums[i][plan] * value(z)
                total_cost += cost
                total_preference += preference_scores[i].get(plan, 0)
                logger.debug(f"Assigned {plan} to {employees[i]}, Cost: {cost:.2f}")
                assigned = True
                break
        if not assigned:
            logger.error(f"Employee {employees[i]} not assigned")
            print(f"Employee {employees[i]} not assigned")
            return None

    # Extract contributions
    for status, comp in contrib_vars:
        contrib = value(contrib_vars[(status, comp)])
        contributions[(status, comp)] = contrib
        logger.debug(f"Contribution for {status}, {comp}: {contrib*100:.1f}%")

    # Validate constraints
    if debug:
        print("Checking if we followed all rules!")
    one_plan_satisfied = all(sum(1 for j in range(num_plans) if value(x[i][j]) > 0.5) == 1 for i in range(num_employees))
    constraint_status['One_Plan_Per_Employee'] = one_plan_satisfied

    available_plans_satisfied = all(plan in premiums[i] for i, (_, plan) in enumerate(assignments))
    constraint_status['Available_Plans_Only'] = available_plans_satisfied

    hsa_assigned = any(plan in hsa_plans for _, plan in assignments)
    constraint_status['HSA_Assigned'] = hsa_assigned
    if not hsa_assigned:
        logger.error("HSA constraint failed")
        print("Error: HSA constraint failed")
        return None

    level3_plans = [plan for plan in plans_list if plan in data['plan_level_id3'].dropna().astype(str).unique()]
    level3_assigned = any(plan in level3_plans for _, plan in assignments) if level3_plans else True
    constraint_status['Level3_Assigned'] = level3_assigned

    if budget is not None:
        budget_satisfied = total_cost <= budget
        constraint_status['Budget_Satisfied'] = budget_satisfied
        logger.debug(f"Budget: {total_cost:.2f} <= {budget:.2f}, {'Satisfied' if budget_satisfied else 'Not satisfied'}")
    else:
        constraint_status['Budget_Satisfied'] = True

    unique_plans = len(set(plan for _, plan in assignments))
    max_plans_satisfied = unique_plans <= 5
    constraint_status['Max_5_Plans'] = max_plans_satisfied

    if fp1_assignments and objective_type == 'preference':
        fp2_differs = any(emp != next(e for e, p in fp1_assignments if e == emp) for emp, plan in assignments)
        constraint_status['FP2_Differs_From_FP1'] = fp2_differs
    else:
        constraint_status['FP2_Differs_From_FP1'] = True

    contrib_bounds_satisfied = all(0.1 <= value(c) <= 1.0 for c in contrib_vars.values())
    constraint_status['Contrib_Bounds'] = contrib_bounds_satisfied

    logger.info(f"Results: Cost ${total_cost:.2f}, Preference {total_preference}, Plans {unique_plans}")
    if debug:
        print(f"Spent ${total_cost:.2f}, happiness score {total_preference}, used {unique_plans} plans!")

    assign_data = [{'Employee': emp, 'Plan': plan} for emp, plan in assignments]
    contrib_data = [{'Status': status, 'Composition': comp, 'Contribution (%)': contrib*100} for (status, comp), contrib in contributions.items()]
    summary_data = {
        'Total Cost': [total_cost],
        'Total Preference Score': [total_preference],
        'Unique Plans': [unique_plans]
    }
    summary_data.update(constraint_status)
    constraint_data = [{'Constraint': k, 'Satisfied': v} for k, v in constraint_status.items()]
    assign_df = pd.DataFrame(assign_data)
    contrib_df = pd.DataFrame(contrib_data)
    summary_df = pd.DataFrame(summary_data)
    constraint_df = pd.DataFrame(constraint_data)
    assign_df.to_excel(excel_writer, sheet_name='Process_Results', index=False, startrow=0)
    contrib_df.to_excel(excel_writer, sheet_name='Process_Results', index=False, startrow=len(assign_data) + 2)
    summary_df.to_excel(excel_writer, sheet_name='Process_Results', index=False, startrow=len(assign_data) + len(contrib_data) + 4)
    constraint_df.to_excel(excel_writer, sheet_name='Process_Results', index=False, startrow=len(assign_data) + len(contrib_data) + len(summary_data) + 6)
    sheet = excel_writer.sheets['Process_Results']
    sheet['A' + str(len(assign_data) + len(contrib_data) + len(summary_data) + len(constraint_data) + 9)] = (
        "Explanation: First table: employee plans. Second: contribution rates. Third: totals. Fourth: rule checks. "
        "It’s like a scoreboard for teams, budgets, and fairness!"
    )
    logger.debug("Saved process_results output")

    result = {
        'assignments': assignments,
        'contributions': contributions,
        'total_cost': total_cost,
        'total_preference': total_preference,
        'unique_plans': unique_plans,
        'constraint_status': constraint_status
    }
    return result

# Run optimization
def run_optimization(budget, objective_type, label, data, employees, num_employees, plans, plan_types, status_composition_pairs, fp1_assignments=None):
    if debug:
        print(f"\n=== {label} Puzzle! ===")
        print(f"Focusing on {'saving money' if objective_type == 'cost' else 'employee happiness'} and setting contribution rates!")
    logger.info(f"Starting {label}")
    if budget is not None:
        logger.info(f"Budget: ${budget:.2f}")
    else:
        logger.info("Budget: None")

    premiums, preferences, preference_scores, employee_groups = process_employee_data(data, plan_types, plans, status_composition_pairs)
    if debug:
        print("Available Plans:", list(plans))
        for i, emp_premiums in enumerate(premiums):
            print(f"  Employee {employees[i]}: {emp_premiums}")

    problem, x, y, contrib_vars, z_vars, plans_list, plan_indices = setup_lp_model(plans, num_employees, status_composition_pairs, objective_type, label)
    set_objective(problem, objective_type, premiums, preference_scores, x, contrib_vars, z_vars, plan_indices, employees, employee_groups)

    hsa_plans = [plan for plan in plans_list if plan in data['hsa_eligible_plan'].dropna().astype(str).unique()]
    add_constraints(problem, x, y, contrib_vars, z_vars, employees, premiums, plan_indices, plans_list, hsa_plans, data, budget, num_employees, len(plans_list), employee_groups, fp1_assignments, objective_type)

    logger.info(f"Problem: {len(problem.variables())} vars, {len(problem.constraints)} constraints")
    if debug:
        print(f"Problem: {len(problem.variables())} vars, {len(problem.constraints)} constraints")

    status = solve_model(problem, label)
    result = process_results(status, x, y, contrib_vars, z_vars, employees, premiums, preference_scores, plans_list, plan_indices, num_employees, len(plans_list), hsa_plans, data, budget, fp1_assignments, objective_type, label, employee_groups)

    if result:
        print("\nEmployee Plan Assignments:")
        for emp_id, plan in result['assignments']:
            print(f"  Employee {emp_id}: {plan}")
        print("\nContribution Percentages:")
        for (status, comp), contrib in result['contributions'].items():
            print(f"  {status}, {comp}: {contrib*100:.1f}%")
        print(f"\nTotal Employer Cost: ${result['total_cost']:.2f}")
        print(f"Total Preference Score: {result['total_preference']}")
        print(f"Number of Unique Plans: {result['unique_plans']}")
        print("Constraint Status:")
        for constraint, satisfied in result['constraint_status'].items():
            print(f"  {constraint}: {'Satisfied' if satisfied else 'Not Satisfied'}")

    if result:
        assign_data = [{'Employee': emp, 'Plan': plan} for emp, plan in result['assignments']]
        contrib_data = [{'Status': status, 'Composition': comp, 'Contribution (%)': contrib*100} for (status, comp), contrib in result['contributions'].items()]
        summary_data = {
            'Total Cost': [result['total_cost']],
            'Total Preference Score': [result['total_preference']],
            'Unique Plans': [result['unique_plans']]
        }
        summary_data.update(result['constraint_status'])
        assign_df = pd.DataFrame(assign_data)
        contrib_df = pd.DataFrame(contrib_data)
        summary_df = pd.DataFrame(summary_data)
        assign_df.to_excel(excel_writer, sheet_name=f'Run_Optimization_{label}', index=False, startrow=0)
        contrib_df.to_excel(excel_writer, sheet_name=f'Run_Optimization_{label}', index=False, startrow=len(assign_data) + 2)
        summary_df.to_excel(excel_writer, sheet_name=f'Run_Optimization_{label}', index=False, startrow=len(assign_data) + len(contrib_data) + 4)
        sheet = excel_writer.sheets[f'Run_Optimization_{label}']
        sheet['A' + str(len(assign_data) + len(contrib_data) + len(summary_data) + 7)] = (
            f"Explanation: First table: {label} plan assignments. Second: contribution rates. Third: totals and rule checks. "
            "It’s the score for this round!"
        )
    else:
        pd.DataFrame({'Status': ['Failed']}).to_excel(excel_writer, sheet_name=f'Run_Optimization_{label}', index=False)
        sheet = excel_writer.sheets[f'Run_Optimization_{label}']
        sheet['A2'] = ("Explanation: Puzzle failed, like missing game pieces.")
    logger.debug(f"Saved run_optimization output for {label}")

    return result

# Get user input
def get_user_input(status_composition_pairs, contributions):
    if debug:
        print("\nAsking for puzzle settings!")
    print("\nEnter budget (or 'no budget' for default):")
    budget_input = input().strip().lower()
    if budget_input == 'no budget':
        budget = None
    else:
        try:
            budget = float(budget_input)
            if budget <= 0:
                raise ValueError("Budget must be positive.")
        except ValueError:
            logger.error("Invalid budget")
            raise ValueError("Invalid budget. Enter a number or 'no budget'.")
    if debug:
        print(f"Budget set to {budget if budget is not None else 'no budget'}!")

    print("Use default contributions? (yes/no):")
    contrib_input = input().strip().lower()
    if contrib_input == 'yes':
        contrib_set = contributions.copy()
        if debug:
            print("Using default contributions!")
    else:
        contrib_set = {}
        print("Enter contribution percentages as decimals (e.g., 0.5 for 50%):")
        for status, comp in status_composition_pairs:
            print(f"Contribution for {status}, {comp}:")
            try:
                contrib = float(input())
                if contrib < 0 or contrib > 1:
                    raise ValueError(f"Contribution for {status}, {comp} must be 0-1.")
                contrib_set[(status, comp)] = contrib
                if debug:
                    print(f"Set {status}, {comp} to {contrib*100}%!")
            except ValueError:
                logger.error(f"Invalid contribution for {status}, {comp}")
                raise ValueError(f"Invalid contribution for {status}, {comp}")

    contrib_data = [{'Status': status, 'Composition': comp, 'Contribution': contrib*100} for (status, comp), contrib in contrib_set.items()]
    input_data = {'Budget': [budget if budget is not None else 'No budget']}
    input_df = pd.DataFrame(input_data)
    contrib_df = pd.DataFrame(contrib_data)
    input_df.to_excel(excel_writer, sheet_name='Get_User_Input', index=False, startrow=0)
    contrib_df.to_excel(excel_writer, sheet_name='Get_User_Input', index=False, startrow=3)
    sheet = excel_writer.sheets['Get_User_Input']
    sheet['A' + str(len(contrib_data) + 6)] = (
        "Explanation: First table: budget. Second: contributions (default or user-set). "
        "It’s like setting money and snack shares!"
    )
    logger.debug("Saved get_user_input output")

    return budget, contrib_set

# Main execution
def main():
    if debug:
        print("\n=== Health Plan Puzzle! ===")
        print("Picking the best plans and contribution rates for employees!")
    logger.info("Starting optimization")

    try:
        data = load_and_validate_data('employees.json')
        employees, num_employees, plans, plan_types, status_composition_pairs = initialize_variables(data)
        contributions = compute_contributions(data, status_composition_pairs)

        budget, contrib_set = get_user_input(status_composition_pairs, contributions)

        if budget is None:
            logger.info("Calculating default budget")
            if debug:
                print("\nNo budget set, so we’re calculating one!")
            budget = 0
            for idx, row in data.iterrows():
                contrib = contrib_set.get((row['memberStatus'], row['composition']), 0.0)
                budget += row['premium'] * contrib
            logger.info(f"Default budget: ${budget:.2f}")
            print(f"Default budget: ${budget:.2f}")

        results = []
        if debug:
            print("\n=== Optimizing for Lowest Cost (fp1) ===")
        fp1_result = run_optimization(budget, 'cost', 'fp1_Lowest_Cost', data, employees, num_employees, plans, plan_types, status_composition_pairs)
        if fp1_result:
            results.append(('fp1_Lowest_Cost', budget, fp1_result))
            fp1_assignments = fp1_result['assignments']
        else:
            fp1_assignments = None

        if debug:
            print("\n=== Optimizing for Highest Preference (fp2) ===")
        fp2_result = run_optimization(budget, 'preference', 'fp2_Highest_Preference', data, employees, num_employees, plans, plan_types, status_composition_pairs, fp1_assignments)
        if fp2_result:
            results.append(('fp2_Highest_Preference', budget, fp2_result))

        print("\n=== Summary of Results ===")
        logger.info("Generating summary")
        if debug:
            print("Final scorecard with plans and contribution rates!")
        summary_data = []
        for label, budget_val, result in results:
            print(f"\nOptimization: {label}")
            logger.info(f"Summary for {label}")
            print("Assignments:")
            for emp_id, plan in result['assignments']:
                print(f"  Employee {emp_id}: {plan}")
            print("Contribution Percentages:")
            for (status, comp), contrib in result['contributions'].items():
                print(f"  {status}, {comp}: {contrib*100:.1f}%")
            if budget_val is not None:
                print(f"Budget: ${budget_val:.2f}")
            else:
                print("Budget: None")
            print(f"Total Cost: ${result['total_cost']:.2f}")
            print(f"Total Preference Score: {result['total_preference']}")
            print(f"Number of Unique Plans: {result['unique_plans']}")
            print("Constraint Status:")
            for constraint, satisfied in result['constraint_status'].items():
                print(f"  {constraint}: {'Satisfied' if satisfied else 'Not Satisfied'}")
            summary_data.append({
                'Optimization': label,
                'Budget': budget_val if budget_val is not None else 'None',
                'Total Cost': result['total_cost'],
                'Total Preference Score': [result['total_preference']],
                'Unique Plans': result['unique_plans'],
                **result['constraint_status']
            })

        summary_df = pd.DataFrame(summary_data)
        contrib_data = []
        for label, _, result in results:
            for (status, comp), contrib in result['contributions'].items():
                contrib_data.append({
                    'Optimization': label,
                    'Status': status,
                    'Composition': comp,
                    'Contribution (%)': contrib*100
                })
        contrib_df = pd.DataFrame(contrib_data)
        summary_df.to_excel(excel_writer, sheet_name='Main_Summary', index=False, startrow=0)
        contrib_df.to_excel(excel_writer, sheet_name='Main_Summary', index=False, startrow=len(summary_data) + 2)
        sheet = excel_writer.sheets['Main_Summary']
        sheet['A' + str(len(summary_data) + len(contrib_data) + 5)] = (
            "Explanation: First table: results for fp1 and fp2. Second: contribution rates. "
            "It’s the final report card!"
        )
        logger.debug("Saved main summary")

        print("\n=== Optimization Complete ===")
        logger.info("Optimization complete")
        if debug:
            print("Puzzle solved! We picked plans and set fair contributions!")

    finally:
        excel_writer.close()
        logger.debug("Closed Excel writer")

if __name__ == "__main__":
    main()

2025-05-04 13:25:22,664 [INFO] Starting optimization
2025-05-04 13:25:22,665 [INFO] Loading data from 'employees.json'...
2025-05-04 13:25:22,673 [INFO] Data loaded successfully.
2025-05-04 13:25:22,676 [INFO] All required columns validated.
2025-05-04 13:25:22,678 [INFO] HSA-eligible plan validation passed.
2025-05-04 13:25:22,686 [DEBUG] Saved load_and_validate_data output to Excel
2025-05-04 13:25:22,693 [INFO] Initialized: 7 employees, 3 status-composition pairs
2025-05-04 13:25:22,695 [DEBUG] Saved initialize_variables output to Excel
2025-05-04 13:25:22,697 [DEBUG] Set default contribution for FT, Employee Only: 85%
2025-05-04 13:25:22,697 [DEBUG] Set default contribution for FT, Employee + Spouse: 75%
2025-05-04 13:25:22,698 [DEBUG] Set default contribution for FT, Employee + Family: 65%
2025-05-04 13:25:22,698 [INFO] Default contributions calculated
2025-05-04 13:25:22,700 [DEBUG] Saved compute_contributions output to Excel



=== Health Plan Puzzle! ===
Picking the best plans and contribution rates for employees!

Hey! We're loading a list of employees and their health plan options from a file called 'employees.json'. This is like getting a class roster with everyone's favorite activities!
Checking if the file has all the info we need, like employee IDs and plan costs!
Ensuring at least one HSA plan exists, like checking for a goalie!

Setting up the basics: employees, plans, and contribution groups!
Found 7 employees and 3 groups for contributions!

Figuring out default company payments for each employee group!
Made a default list of how much the company pays for each group!

Asking for puzzle settings!

Enter budget (or 'no budget' for default):
Budget set to no budget!
Use default contributions? (yes/no):


2025-05-04 13:25:29,135 [DEBUG] Saved get_user_input output
2025-05-04 13:25:29,138 [INFO] Calculating default budget
2025-05-04 13:25:29,141 [INFO] Default budget: $7720.26
2025-05-04 13:25:29,141 [INFO] Starting fp1_Lowest_Cost
2025-05-04 13:25:29,142 [INFO] Budget: $7720.26
2025-05-04 13:25:29,143 [INFO] Processing employee data
2025-05-04 13:25:29,145 [DEBUG] Employee 3: status=FT, composition=Employee Only
2025-05-04 13:25:29,145 [DEBUG]   Added plan 58081GA0010030 with premium 451.92
2025-05-04 13:25:29,146 [DEBUG]   Added plan 60224GA0010016 with premium 584.77
2025-05-04 13:25:29,147 [DEBUG]   Added plan 58081GA0010078 with premium 377.60
2025-05-04 13:25:29,147 [DEBUG]   Added plan 58081GA0010075 with premium 376.57
2025-05-04 13:25:29,148 [DEBUG]   Added plan 58081GA0010079 with premium 379.34
2025-05-04 13:25:29,149 [DEBUG]   Plan 58081GA0010030 score: 5
2025-05-04 13:25:29,149 [DEBUG]   Plan 60224GA0010016 score: 10
2025-05-04 13:25:29,150 [DEBUG]   Plan 58081GA0010078 scor

Using default contributions!

No budget set, so we’re calculating one!
Default budget: $7720.26

=== Optimizing for Lowest Cost (fp1) ===

=== fp1_Lowest_Cost Puzzle! ===
Focusing on saving money and setting contribution rates!

Checking each employee’s plan options, costs, and preferences!
Done! We know each employee’s plan costs and preferences!
Available Plans: ['58081GA0010075', '58081GA0010030', '60224GA0010016', '15105GA0020010', '45495GA0050009', '45495GA0040009', '58081GA0010079', '82824GA0110039', '49046GA0700006', '58081GA0010078']
  Employee 3: {'58081GA0010030': 451.92, '60224GA0010016': 584.77, '58081GA0010078': 377.6, '58081GA0010075': 376.57, '58081GA0010079': 379.34}
  Employee 9: {'58081GA0010030': 585.17, '49046GA0700006': 688.81, '15105GA0020010': 701.75}
  Employee 16: {'58081GA0010030': 1523.07, '82824GA0110039': 1791.8}
  Employee 17: {'58081GA0010030': 1146.61, '58081GA0010075': 955.45, '49046GA0700006': 1349.71}
  Employee 18: {'58081GA0010030': 1567.49, '45495G

2025-05-04 13:25:29,345 [DEBUG] Saved setup_lp_model output for fp1_Lowest_Cost
2025-05-04 13:25:29,351 [INFO] Constructing objective (cost)...
2025-05-04 13:25:29,360 [DEBUG]   Employee 3, Plan 58081GA0010030, Premium 451.92
2025-05-04 13:25:29,363 [DEBUG]   Employee 3, Plan 60224GA0010016, Premium 584.77
2025-05-04 13:25:29,366 [DEBUG]   Employee 3, Plan 58081GA0010078, Premium 377.60
2025-05-04 13:25:29,369 [DEBUG]   Employee 3, Plan 58081GA0010075, Premium 376.57
2025-05-04 13:25:29,371 [DEBUG]   Employee 3, Plan 58081GA0010079, Premium 379.34
2025-05-04 13:25:29,373 [DEBUG]   Employee 9, Plan 58081GA0010030, Premium 585.17
2025-05-04 13:25:29,374 [DEBUG]   Employee 9, Plan 49046GA0700006, Premium 688.81
2025-05-04 13:25:29,376 [DEBUG]   Employee 9, Plan 15105GA0020010, Premium 701.75
2025-05-04 13:25:29,378 [DEBUG]   Employee 16, Plan 58081GA0010030, Premium 1523.07
2025-05-04 13:25:29,378 [DEBUG]   Employee 16, Plan 82824GA0110039, Premium 1791.80
2025-05-04 13:25:29,379 [DEBUG] 

Made a grid for 7 employees, 10 plans, 3 contribution rates, and 210 cost helpers!

Deciding our goal: save money or make employees happy, while setting contribution rates!
We’re trying to spend as little as possible while picking plans and contribution rates!

Adding rules to keep our assignments and contributions fair!
Rule 1: Every employee gets one plan!
Rule 2: Only assign available plans!
Rule 3: At least one HSA plan!
Rule 4: At least one Level3 plan if available!
Rule 5: Stay within budget using contribution rates!
Rule 6: Use up to 5 different plans!
Rule 8: Contributions between 10% and 100%!
Rule 9: Link contributions and assignments for costs!
Problem: 153 vars, 207 constraints

Solving the puzzle with a smart computer trick!


2025-05-04 13:25:29,555 [INFO] Solver status: 1, time: 0.15s
2025-05-04 13:25:29,557 [DEBUG] Saved solve_model output for fp1_Lowest_Cost
2025-05-04 13:25:29,558 [INFO] Processing assignments and contributions
2025-05-04 13:25:29,558 [DEBUG] Assigned 60224GA0010016 to 3, Cost: 58.48
2025-05-04 13:25:29,559 [DEBUG] Assigned 58081GA0010030 to 9, Cost: 58.52
2025-05-04 13:25:29,559 [DEBUG] Assigned 58081GA0010030 to 16, Cost: 152.31
2025-05-04 13:25:29,560 [DEBUG] Assigned 58081GA0010075 to 17, Cost: 95.55
2025-05-04 13:25:29,561 [DEBUG] Assigned 45495GA0040009 to 18, Cost: 156.18
2025-05-04 13:25:29,563 [DEBUG] Assigned 58081GA0010075 to 19, Cost: 81.57
2025-05-04 13:25:29,563 [DEBUG] Assigned 45495GA0040009 to 30, Cost: 353.73
2025-05-04 13:25:29,564 [DEBUG] Contribution for FT, Employee Only: 10.0%
2025-05-04 13:25:29,564 [DEBUG] Contribution for FT, Employee + Spouse: 10.0%
2025-05-04 13:25:29,564 [DEBUG] Contribution for FT, Employee + Family: 10.0%
2025-05-04 13:25:29,567 [DEBUG] Bu

Welcome to the CBC MILP Solver 
Version: 2.10.12 
Build Date: Mar  5 2025 

command line - cbc /var/folders/6n/w_nclz597sq6klys0_6tskrm0000gn/T/067f6f14bb6f432d85ab7925e4d66741-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/6n/w_nclz597sq6klys0_6tskrm0000gn/T/067f6f14bb6f432d85ab7925e4d66741-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 212 COLUMNS
At line 871 RHS
At line 1079 BOUNDS
At line 1166 ENDATA
Problem MODEL has 207 rows, 153 columns and 475 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.00 seconds
Cgl0002I 47 variables fixed
Cgl0003I 2 fixed, 0 tightened bounds, 25 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 15 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 6 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 6 strengthened rows, 0 substitutions
Cgl0003